In [1]:
import os

os.chdir("../..")
os.getcwd()

'/home/INTEXSOFT/maksim.stupakevich/work/ds_practice'

In [28]:
import pickle
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Levenshtein import editops
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [3]:
train_df = pd.read_csv("data/enzyme/train.csv", index_col="seq_id")

In [4]:
updated_df = pd.read_csv("data/enzyme/train_updates_20220929.csv", index_col="seq_id")

In [5]:
updated_df = updated_df.drop(["data_source"], axis=1).dropna()

In [6]:
train_df.update(updated_df)

In [7]:
train_df

,protein_sequence,pH,data_source,tm
seq_id,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...
31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6


In [8]:
def get_count(groups):
    count = 0
    for k, v in groups.items():
        count += len(v)
    return count

In [144]:
def group(df):
    groups = defaultdict(list)
    proteins = df["protein_sequence"].unique().tolist()
    with tqdm(total=len(proteins)) as pbar:
        while proteins:
            protein = proteins.pop(0)
            idxs_to_del = []
            for i, el in enumerate(proteins):
                similarity = editops(protein[:20], el[:20])
                if len(similarity) <= 5:
                    groups[protein].append(el)
                    idxs_to_del.append(i)
            [proteins.pop(el) for el in list(reversed(idxs_to_del))]
            pbar.update(len(groups[protein]) + 1)

    return {i: [k, *v] for i, (k, v) in enumerate(groups.items())}

In [145]:
groups = group(train_df)

100%|█████████████████████████████████████| 28981/28981 [10:09<00:00, 47.58it/s]


In [146]:
get_count(groups)

28981

In [147]:
groups

{0: ['AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR'],
 1: ['AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSRLQAQRRAQRVAWEDGDENVGQTVIPAQEEEGIEKPAEVHPTGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLKEEQKEEEERKAQEEQARREHEEYLKLKEAFVVEEEGVSETMTEEQSHSFLTEFINYIKKSKVVLLEDLAFQMGLRTQDAINRIQDLLTEGTLTGVIDDRGKFIYITPEELAAVANFIRQRGRVSITELAQASNSLISWGQDLPAQAS'],
 2: ['AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYATLGVAKNANGKDIKKAYYQLAKKYHPDTNKEDPDAGRKFQEVSEAYEVLSDEQKRREYDTYGQTAENIGRQGGGFPGGGAGGFGPEGFSQSWQFRSSIDPEELFRKIFGEGNFRTNSFDDFADSKFGFGQAQEMVMDLTFAQAARGVNKDVNVNVVDQCPKCAGTKCEPGTKPGRCQYCNGTGFETVSTGPFVMRSTCRYCQGTRQHIKYPCSECEGKGRTVQRRKVTVPVPAGIENGQTVRMQVGSKELFVTFRVERSDYFRREGADVHTDAAISLAQAVLGGTVRVQGVYEDQWINVEP

In [148]:
len(groups)

23185

In [149]:
len({k: v for k, v in groups.items() if len(v) > 1})

1343

In [150]:
with open("data/enzyme/train_groups_3_all.pkl", "wb") as f:
    pickle.dump(groups, f)

In [151]:
with open("data/enzyme/train_groups_3.pkl", "wb") as f:
    pickle.dump({k: v for k, v in groups.items() if len(v) > 1}, f)

In [152]:
def get_group(protein, groups):
    for group_num, group_proteins in groups.items():
        if protein in group_proteins:
            return group_num
    return -1

In [153]:
train_df["group"] = train_df["protein_sequence"].apply(
    lambda x: get_group(
        x,
        {i: v for i, (k, v) in enumerate({k: v for k, v in groups.items() if len(v) > 1}.items())},
    )
)

In [154]:
train_df[train_df["group"] != -1].head()

,protein_sequence,pH,data_source,tm,group,wildtype
seq_id,,,,,,
5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4,0,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
6,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,0,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
20,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...,7.0,doi.org/10.1038/s41592-020-0801-4,42.5,1,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...
31,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...,7.0,doi.org/10.1038/s41592-020-0801-4,40.8,2,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...
40,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,3,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...


In [157]:
def get_wildtypes(groups, method="mode"):
    wildtypes = {}
    if method == "mode":
        for group_num, group_proteins in groups.items():
            wildtypes[group_num] = stats.mode(groups[group_num])[0][0]
    elif method == "editops":
        for group_num, group_proteins in groups.items():
            differenсies = {}
            for cur_protein in group_proteins:
                differenсies[cur_protein] = 0
                for protein in group_proteins:
                    differenсies[cur_protein] += len(editops(cur_protein, protein))
            wildtypes[group_num] = min(differenсies, key=differenсies.get)
    else:
        raise TypeError
    return wildtypes

In [159]:
wildtypes = get_wildtypes(
    {i: v for i, (k, v) in enumerate({k: v for k, v in groups.items() if len(v) > 1}.items())},
    method="editops",
)

In [162]:
train_df["wildtype"] = train_df.apply(
    lambda x: wildtypes[x["group"]] if x["group"] != -1 else np.nan,
    axis=1,
)

In [163]:
train_df.head()

,protein_sequence,pH,data_source,tm,group,wildtype
seq_id,,,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,-1,NaN
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,-1,NaN
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,-1,NaN
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,-1,NaN
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,-1,NaN


In [164]:
train_df[train_df["group"] != -1].head()

,protein_sequence,pH,data_source,tm,group,wildtype
seq_id,,,,,,
5,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4,0,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
6,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,0,AACFWRRTVIPKPPFRGISTTSARSTVMPAWVIDKYGKNEVLRFTQ...
20,AAMNPEYDYLFKLLLIGDSGVGKSCLLLRFADDTYTESYISTIGVD...,7.0,doi.org/10.1038/s41592-020-0801-4,42.5,1,MNPEYDYLFKLLLIGDSGVGKSCLLLRFADDSYLDSYISTIGVDFK...
31,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...,7.0,doi.org/10.1038/s41592-020-0801-4,40.8,2,AATSVFCGSSHRQLHHAVIPHGKGGRSSVSGVVATVFGATGFLGRY...
40,ACPDSNDKEIRGFCFKFVVQKMAYNDARNWCHYQNPVGPSYLAVVG...,7.0,doi.org/10.1038/s41592-020-0801-4,45.7,3,ACPDSNDKEIRGFCFKFVVQKMTYNDARNWCHYQNPVGPSYLAVVG...


In [166]:
train_df.to_csv("data/enzyme/train_with_wildtypes.csv", index=False)

In [173]:
editops(train_df.loc[28856, "wildtype"], train_df.loc[28856, "protein_sequence"])

[('replace', 28, 28)]

In [172]:
train_df[train_df["group"] == 1286]

,protein_sequence,pH,data_source,tm,group,wildtype
seq_id,,,,,,
28792,MVLSEGEWALVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,4.2,NaN,4.0,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28793,MVLSEGEWGLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,4.2,NaN,4.0,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28794,MVLSEGEWQAVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,9.6,10.1002/pro.5560020704,81.4,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28795,MVLSEGEWQLVAHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,9.6,10.1002/pro.5560020704,81.4,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28796,MVLSEGEWQLVLHAWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,5.0,NaN,25.0,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
...,...,...,...,...,...,...
28852,MVLSEGEWQLVLHVWAKVEADVAGHGQDIVIRLFKSHPETLEKFDR...,11.0,10.1021/bi00210a011,70.5,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28853,MVLSEGEWQLVLHVWAKVEADVAGHGQDLLIRLFKSHPETLEKFDR...,11.0,10.1021/bi00210a011,74.8,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...
28854,MVLSEGEWQLVLHVWAKVEADVAGHGQDMLIRLFKSHPETLEKFDR...,11.0,10.1021/bi00210a011,74.8,1286,MVLSEGEWQLVLHVWAKVEADVAGHGQDALIRLFKSHPETLEKFDR...


In [95]:
train_df.loc[28792, "protein_sequence"]

'MVLSEGEWALVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKTEAEMKASEDLKKHGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGDFGADAQGAMNKALELFRKDIAAKYKELGYQG'